This notebook explains and demonstrates the use of the search_phase_space function. All coordinate transformations are dervied from Bovy (2011). [https://github.com/jobovy/stellarkinematics/blob/master/stellarkinematics.pdf]

To convert from ICRS coordinates to rectangular galactic coordinates, the function first fetches the ra and dec of the north galactic pole, $\alpha_{\textrm{NGP}}$ and $\delta_{\textrm{NGP}}$ in ICRS using galpy. These are used to calculate the sine and cosine of the galactic parallactic angle $\phi$ for each star, according to the equations:

$$\cos \phi = \frac{\sin \delta_{\textrm{NGP}} - \sin \delta \sin b}{\cos \delta \cos b}$$

$$\sin \phi = \frac{\sin(\alpha - \alpha_{\textrm{NGP}})\cos \delta_{\textrm{NGP}}}{\cos b}$$

Where $\alpha$ and $\delta$ are the ra and dec of the star whose coordinates are being converted, and $b$ is its galactic latitude. These are then used to calculate the star's proper motion in $b$ and in its galactic longitude $l$ according to the equation:

$$\mu_{l} \cos b = \mu_{\alpha} \cos{\delta} \cos{\phi} + \mu_{\delta} \sin{\phi}$$

$$\mu_{b} = \mu_{\delta} \cos{\phi} - \mu_{\alpha} \cos{\delta} \sin{\phi}$$

Note that in the code, $\textrm{pmra} \equiv \mu_{\alpha} \cos{\delta}$ and $\textrm{pml} \equiv \mu_{l} \cos b$.

After this, the program calculates $x$, $y$, and $z$ in galactic rectangular coordinates according to:

$$x = d \cos b \cos l$$

$$y = d \cos b \sin l$$

$$z = d \sin b$$

Where $d$ is the distance to the star. Taking the time derivative of these equations allows us to calculate $v_x$, $v_y$, and $v_z$:

$$v_x = v_r \cos b \cos l - d \mu_b \sin b \cos l - d \mu_l \cos b \sin l$$

$$v_y = v_r \cos b \sin l - d \mu_b \sin b \sin l + d \mu_l \cos b \cos l$$

$$v_z = v_r \sin b + d \mu_b \cos b$$

Where $d$ is the distance to the star. In the code, a conversion factor $k$ is used in the velocity equations to convert $\textrm{kpc mas/yr}$ to $\textrm{km/s}$. Once the units have been converted, the query checks for stars satisfying the equation:

$$\sqrt{(x-x_o)^2 + (y-y_0)^2 + (z-z_0)^2 + (A(v_x - v_{x_0}))^2 + (A(v_y - v_{y_0}))^2 + (A(v_z - v_{z_0}))^2} < \epsilon$$

Where $A$ is a scale factor that can be adjusted to weight the velocities similarly to the positions. Example usages are below.

In [1]:
from search_phase_space import *

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [2]:
help(search_phase_space)

Help on function search_phase_space in module search_phase_space:

search_phase_space(x, y, z, vx, vy, vz, epsilon, v_scale=1.0, cone_r=1.0)
    NAME:
        search_phase_space
        
    PURPOSE:
        query the Gaia DR2 RV catalogue for stars near a point in phase space
        
    INPUT:
        x - rectangular x coordinate in the galactic frame (can be Quantity,
        otherwise given in kpc)
        
        y - rectangular y coordinate in the galactic frame (can be Quantity,
        otherwise given in kpc)
        
        z - rectangular z coordinate in the galactic frame (can be Quantity,
        otherwise given in kpc)
        
        vx - x velocity in the galactic frame (can be Quantity, otherwise given
        in km/s)
        
        vy - y velocity in the galactic frame (can be Quantity, otherwise given
        in km/s)
        
        vz - z velocity in the galactic frame (can be Quantity, otherwise given
        in km/s)
        
        epsilon - radius in ph

In [3]:
table = search_phase_space(1,1,1,1,1,1,10)
table['x','y','z','vx','vy','vz']

Query finished.


x,y,z,vx,vy,vz
float64,float64,float64,float64,float64,float64
0.3248547285099525,0.3279740455677438,0.3153034498744913,-0.29035230958035463,-5.1796266147947,-4.353165870439843
0.2188858173997254,0.2178464318294424,0.22580041586685468,-1.275905453220787,3.428241380577102,-1.4298321419533973
0.12780404047322055,0.12814092607639407,0.13156138199108242,1.046825144670162,3.4067004099763505,-1.123290133914609
0.2843689575919812,0.2885781542557433,0.29174504457791295,-4.331893530634634,2.6449143076819963,6.045735212272312


We will get different results if we adjust v_scale or epsilon.

In [4]:
table = search_phase_space(1,1,1,1,1,1,10,v_scale=0.01)
len(table)

Query finished.


300

In [5]:
table = search_phase_space(1,1,1,1,1,1,100)
len(table)

Query finished.


241

We can also change the initial search radius. This will allow us to check more stars, but will slow down the query.

In [6]:
%time table0 = search_phase_space(1,1,1,1,1,1,10)
%time table1 = search_phase_space(1,1,1,1,1,1,10, cone_r = 5.0)
%time table2 = search_phase_space(1,1,1,1,1,1,10, cone_r = 10.0)

Query finished.
Wall time: 7.68 s
Query finished.
Wall time: 9.6 s
Query finished.
Wall time: 17.6 s
